In [1]:
import numpy as np
import collections
import pickle
import os
import argparse
from openbabel import openbabel as ob
from openbabel import pybel
import pandas as pd
import time
import sys

sys.path.append("..")

from analyses import analysis
from schnetpack import Properties
from multiprocessing import Process, Queue
from ase import Atoms
from ase.db import connect

sys.path.append('..')
sys.path.append('../analyses')

import analysis
import datatypes
import qm9_filter_generated
from utility_classes import Molecule, ConnectivityCompressor
from utility_functions import run_threaded, print_atom_bond_ring_stats, update_dict

/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


In [3]:
with open('generated_molecules_statistics.pkl', 'rb') as f:
    stats_df = pickle.load(f)

In [8]:
stats_df.keys()

dict_keys(['train_eval_final', 'val_eval_final', 'test_eval_final', 'generated_stats'])

In [9]:
stats_df['generated_stats']

,n_atoms,valid_mol,valid_atoms,C,N,O,F,H,H1C,H1N,...,N1F,O1O,O1F,R3,R4,R5,R6,R7,R8,R>8
0,23.0,0.0,21.0,10.0,0.0,3.0,0.0,10.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,16.0,0.0,14.0,8.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,18.0,0.0,16.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,34.0,0.0,15.0,15.0,2.0,4.0,0.0,13.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16.0,1.0,16.0,7.0,0.0,1.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,34.0,0.0,19.0,17.0,0.0,0.0,0.0,17.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,26.0,0.0,19.0,11.0,3.0,0.0,0.0,12.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61,17.0,1.0,17.0,8.0,0.0,1.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62,36.0,0.0,24.0,15.0,3.0,4.0,0.0,14.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
